In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
lichess_key = os.getenv('LICHESS_KEY')
lichess_key

'lip_9J9hHE4WhMf5AvIVexDf'

In [4]:
import berserk
import threading  # To handle multiple games concurrently



session = berserk.TokenSession(lichess_key)
client = berserk.Client(session)

import berserk
import time
from stockfish import Stockfish
import chess
from tokenizer import ChessTokenizer
import torch
import torch.nn as nn
from model import ChessGPTModel
from engine import Engine
from transformers import GPT2Config


ModuleNotFoundError: No module named 'berserk'

In [12]:
stockfish = Stockfish("/opt/homebrew/bin/stockfish")  # Adjust path to your Stockfish installation
mps_device = torch.device("mps")
stockfish.set_elo_rating(100)

In [13]:
hidden_size = 512
seq_len = 256
n_layer = 12
n_head = 8
tok = ChessTokenizer()
vocab_size = tok.vocabulary_size() #1971
config = GPT2Config(vocab_size=vocab_size, n_positions=seq_len, n_ctx=seq_len, n_embd=hidden_size, n_layer=n_layer, n_head=n_head)

engine = Engine("./modelv1.pth", config)

In [14]:
def play_game(game_id):
    print(f"Playing game {game_id}")
    # Listen to events within the game
    for event in client.bots.stream_game_state(game_id):
        print(event)
        if event['type'] == 'gameFull' or event['type'] == 'gameState':
            
            if event['type'] == 'gameFull':
                # Get the initial board position
                moves = event['state']['moves']
            else:
                moves = event['moves']

            moves = moves.split()

            our_turn = 0
            # if event[""]

            if len(moves) % 2 == 0:  # It's our turn if the move list is even
                # Set the board position for Stockfish

                print(moves)
                
                stockfish.set_position(moves)
                # Get the best move from Stockfish
                best_move = stockfish.get_best_move()

                # next_movee = engine.get_next_move(moves)

                print(best_move)

                print(f"Playing move {best_move}")

                # Play the move on Lichess
                client.bots.make_move(game_id, best_move)

In [15]:
while True:

    print("start loop")

    for event in client.bots.stream_incoming_events():
        # print(event)

        if event['type'] == 'challenge':
            print("challenge_incoming")
            challenge_id = event['challenge']['id']
            client.bots.accept_challenge(challenge_id)

            # print("challenge_accepted")
            # print(event['challenge']['id'])
            thread = threading.Thread(target=play_game, args=(f"{event['challenge']['id']}", ))
            thread.start()





    time.sleep(1)

start loop
challenge_incoming
Playing game pQLAJDPc
{'id': 'pQLAJDPc', 'variant': {'key': 'standard', 'name': 'Standard', 'short': 'Std'}, 'speed': 'correspondence', 'perf': {'name': 'Correspondence'}, 'rated': False, 'createdAt': datetime.datetime(2024, 12, 27, 20, 46, 46, 275000, tzinfo=datetime.timezone.utc), 'white': {'id': 'srikar_bot', 'name': 'srikar_bot', 'title': 'BOT', 'rating': 2000, 'provisional': True}, 'black': {'id': 'tallurium', 'name': 'tallurium', 'title': None, 'rating': 1500, 'provisional': True}, 'initialFen': 'startpos', 'type': 'gameFull', 'state': {'type': 'gameState', 'moves': '', 'wtime': 2147483647, 'btime': 2147483647, 'winc': 0, 'binc': 0, 'status': 'started'}}
[]
c2c4
Playing move c2c4
{'type': 'gameState', 'moves': 'c2c4', 'wtime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'btime': datetime.datetime(1970, 1, 25, 20, 31, 23, 647000, tzinfo=datetime.timezone.utc), 'winc': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=dat

KeyboardInterrupt: 